# [Chialisp Modules](https://chialisp.com/operators)
`mod`, `include`, `defun`, `defun-inline`, `lambda`, `defmacro`, `defconstant`

## `defun` vs `defmacro`
[What is the difference between defn and defmacro?](https://stackoverflow.com/questions/3667403/what-is-the-difference-between-defn-and-defmacro)

> The difference between functions and macros is that on a function call first the arguments of the function are evaluated then the body of the function is evaluated using the arguments.

> Macros on the other hand describe a transformation from one piece of code to another. Any evaluation takes place after the transformation.

> This means that arguments may be evaluated multiple times or not at all. As an example or is a macro. If the first argument of or is false, the second argument will never be evaluated. If or were a function, this would not be possible, because the arguments would always be evaluated before the function runs.

### [Macros](https://github.com/Chia-Network/clvm_tools/blob/main/README.md#macros)
> You can also define macros within a module, which act as inline functions. When a previously defined macro operator is encountered, it "rewrites" the existing statement using the macro, passing along the arguments as literals (ie. they are not evaluated).

- [`qq` and `unquote`](../quote/README.md)
- [defmacro, qq, unquote](https://chialisp.com/examples#defmacro)
- [macros.clvm](https://github.com/Chia-Network/clvm_tools/blob/main/clvm_runtime/macros.clvm)
- [utility_macros.clib](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/utility_macros.clib)


## ChatGPT
> In the programming language Lisp, a macro is a feature that allows programmers to extend the language by defining new syntax. Macros are similar to functions, but they operate at a higher level of abstraction, allowing them to manipulate the code itself rather than just the data.

> Lisp macros are defined using the macro keyword, followed by the name of the macro and its parameters. The macro definition consists of a set of rules for transforming the code, which is written in the same Lisp syntax used for regular functions. When the macro is invoked in the code, these rules are applied to the arguments, generating new code that is then executed.

> For example, a macro might be used to define a new control structure, such as a "foreach" loop that iterates over a list of elements. This can make the code more readable and easier to write, and can also be used to optimize the generated code for better performance.

> Macros are a powerful and flexible tool, but they can also be difficult to use and debug. As a result, they are typically reserved for advanced Lisp programmers who have a deep understanding of the language and its capabilities.

## Chialisp Macro
- Similar to functions, but they operate at a higher level of abstraction, allowing them to manipulate the code itself rather than just the data.
- Can perform complex transformation not just text replacing.
- Define using regular Chialisp syntax and can be called like function

- Similar to C/C++ macro as it's expanded during compiled time?
- Macro may or maynot faster than function.


In [1]:
clsp = Path('macros.clib').read_text()
print_clsp(clsp)

   1 (                                                                                                             
   2     ; "assert" is a macro that wraps repeated instances of "if"                                               
   3     ; usage: (assert A0 A1 ... An R)                                                                          
   4     ; all of A0, A1, ... An must evaluate to non-null, or an exception is raised                              
   5     ; return the value of R (if we get that far)                                                              
   6     (defmacro assert items                                                                                    
   7         (if (r items)                                                                                         
   8             (list if (f items) (c assert (r items)) (q . (x)))                                                
   9             (f items)                                                                                         
  10         )                                                                                                     
  11     )                                                                                                         
  12                                                                                                               
  13     ;; boolean and macro                                                                                      
  14     ;; This lets you write something like (if (and COND1 COND2 COND3) (do-something) (do-something-else))     
  15     (defmacro and ARGS                                                                                        
  16         (if ARGS                                                                                              
  17             (qq (if (unquote (f ARGS))                                                                        
  18                 (unquote (c and (r ARGS)))                                                                    
  19                 ()                                                                                            
  20                 ))                                                                                            
  21             1)                                                                                                
  22     )                                                                                                         
  23                                                                                                               
  24     ;; boolean or macro                                                                                       
  25     ;; This lets you write something like (if (or COND1 COND2 COND3) (do-something) (do-something-else))      
  26     ;; Defines a macro that can manually structure the source code it is replaced with.                       
  27     ;; Allows for advanced compile time behavior.                                                             
  28     (defmacro or ARGS                                                                                         
  29         (if ARGS                                                                                              
  30             (qq (if (unquote (f ARGS))                                                                        
  31                 1                                                                                             
  32                 (unquote (c or (r ARGS)))                                                                     
  33             ))                                                                                                
  34         0)                                                                                                    
  35     )                                              

In [2]:
cons_a_fun = load_program("cons_a-fun.clsp", ["."])
print_program(cons_a_fun)

(a (q 2 2 (c 2 (c 3 ()))) (c (q 4 (q . 97) 5) 1))                                                                  

In [3]:
cons_a_macro = load_program("cons_a-macro.clsp", ["."])
print_program(cons_a_macro)

(c (q . 97) 1)                                                                                                     